In [1]:
import multiprocessing
multiprocessing.set_start_method("spawn", force=True)
print("Number of processors: ", multiprocessing.cpu_count())

Number of processors:  12


In [7]:
from julia.api import Julia 
jpath = '/home/marktas/.local/bin/julia'
jl = Julia(runtime=jpath, compiled_modules=False)

In [9]:
from julia import Main as jl    
from julia import BioSimulator

In [10]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [4]:
import os
from multiprocessing import cpu_count
# read num of cpus and set the julia threas var
os.environ["JULIA_NUM_THREADS"] = str(cpu_count())

#import (py)julia
from julia.api import Julia 
jpath = '/home/marktas/.local/bin/julia'
jl = Julia(runtime=jpath, compiled_modules=False)
from julia import Main as jl
from julia import BioSimulator as bs 

In [9]:
model = bs.Network("multi")  

initial = 2

model <= bs.Species("SS",int(initial))
model <= bs.Species("L1",0)
model <= bs.Species("L2",0)
model <= bs.Species("R1",0)
model <= bs.Species("R2",0)
model <= bs.Species("D",0)

kf_dl = 4e8
nonspecific_correction = 1/5

sliding_correction = 1/2
kf_sliding = 1e7 * sliding_correction

Na = 6.023e23

kf_ssl1 = kf_dl*nonspecific_correction
kb_ssl1 = 22.920792678077465 

kf_ssl2 = kf_dl*nonspecific_correction
kb_ssl2 = 0.007808225837362128

kf_ssr2 = kf_dl*nonspecific_correction
kb_ssr2 = 0.0626785751957368

kf_ssr1 = kf_dl*nonspecific_correction
kb_ssr1 = 183.99091641848813

kf_ssD = kf_dl*nonspecific_correction
kb_ssD = 6.902831840829526e-08

kf_l1l2 = kf_sliding
kb_l1l2 = 3406.6124793451313*sliding_correction

kf_l2D = kf_sliding
kb_l2D = 88.40461309148736*sliding_correction

kf_r1r2 = kf_sliding
kb_r1r2 = 3406.6124793451313*sliding_correction

kf_r2D = kf_sliding
kb_r2D = 11.01306438328072*sliding_correction

model <= bs.Reaction("ss-l1_f",  kf_ssl1, "SS + SS --> L1")
model <= bs.Reaction("ss-l1_b",  kb_ssl1, "L1 --> SS + SS")
model <= bs.Reaction("ss-l2_f",  kf_ssl2, "SS + SS --> L2")
model <= bs.Reaction("ss-l2_b",  kb_ssl2, "L2 --> SS + SS")
model <= bs.Reaction("ss-r2_f",  kf_ssr2, "SS + SS --> R2")
model <= bs.Reaction("ss-r2_b",  kb_ssr2, "R2 --> SS + SS")
model <= bs.Reaction("ss-r1_f",  kf_ssr1, "SS + SS --> R1")
model <= bs.Reaction("ss-r1_b",  kb_ssr1, "R1 --> SS + SS")
model <= bs.Reaction("ss-D_f",   kf_ssD, "SS + SS --> D")
model <= bs.Reaction("ss-D_b",   kb_ssD, "D --> SS + SS")
model <= bs.Reaction("l1-l2_f",  kf_l1l2, "L1 --> L2")
model <= bs.Reaction("l1-l2_b",  kb_l1l2, "L2 --> L1")
model <= bs.Reaction("l2-D_f",   kf_l2D, "L2 --> D")
model <= bs.Reaction("l2-D_b",   kb_l2D, "D --> L2")
model <= bs.Reaction("r1-r2_f",  kf_r1r2, "R1 --> R2")
model <= bs.Reaction("r1-r2_b",  kb_r1r2, "R2 --> R1")
model <= bs.Reaction("r2-D_f",   kf_r2D, "R2 --> D")
model <= bs.Reaction("r2-D_b",   kb_r2D, "D --> R2")

Nmonte = 50000
runtime = 1e-5

simresults = [bs.simulate(model, jl.Direct(),tfinal=runtime) for i in range(Nmonte)]
results = [{'c':jl.hcat(*simresults[i].u).__array__(), 't':simresults[i].t} for i in range(Nmonte)]
# simresults = juliacall.VectorValue
# for i in range(Nmonte):
#     jl.append(simresults,jl.simulate(model, jl.Direct(),tfinal=runtime))


AttributeError: 'list' object has no attribute 'u'

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#definitive routine for mfpis is detailed below

index = 2

results[index]['c']
results[index]['c'][-1]
results[index]['t']

fpi = np.argmax(results[index]['c'][-1] == 1)

fpi, results[index]['t'][fpi]

array([[2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 1]])

array([0, 0, 0, 1, 0, 1, 0, 1, 1])

9-element Vector{Float64}:
 0.0
 2.372237859220639e-9
 1.4628208175078858e-7
 3.001033672771653e-7
 0.005699058104488336
 0.005699231462495758
 0.009411363406971164
 0.009411431725315063
 0.01

(3, 3.001033672771653e-07)

In [ ]:
# here i will keep the fastest routine i found for conversion
for sim in range(50000):
    # a = pd.DataFrame(simresults[sim].u)
    a = simresults[sim].u.__array__()


In [ ]:
@parfor(range(5000))
def fun(i):
    i = pd.concat([i, pd.DataFrame(simresults[sim].u)])

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
a

,0,1,2,3,4,5
0,2,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
0,2,0,0,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1
0,2,0,0,0,0,0


In [ ]:
pd.DataFrame(simresults[0].u)

,0,1,2,3,4,5
0,2,0,0,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1


In [ ]:
type(simresults[1].to_numpy())
#NO NEED TO CONVERT IT TO PYTHON JUST CALL IT AS BELOW 
simresults[1].u[-1]

numpy.ndarray

6-element Vector{Int64}:
 0
 0
 0
 0
 0
 1

In [ ]:
def first(array, condition = lambda x: x == 1):
    for i in range(len(array)):
        if condition(array[i]):
            return i

In [ ]:
pd.DataFrame(simresults[0].u)
x=first(simresults[0].u[-1])
simresults[0].t

,0,1,2,3,4,5
0,2,0,0,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1


4-element Vector{Float64}:
 0.0
 5.3295782525917545e-9
 2.2212434133477408e-7
 0.01

In [ ]:
index = 5
type(simresults[index])
len(simresults[index])
np.shape(simresults[index].__array__())
np.shape(simresults[index])

juliacall.ArrayValue

3

(6, 3)

(6, 3)